In [1]:
import os
os.chdir("../")
%pwd

'f:\\interview\\End-to-End-MLOps-project-with-MLFlow'

In [4]:
import pandas as pd
data = pd.read_csv("artifacts\data_ingestion\winequality-red.csv")
data.head()


<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\giova\AppData\Local\Temp\ipykernel_14596\2260983021.py:2: SyntaxWarning: invalid escape sequence '\d'
  data = pd.read_csv("artifacts\data_ingestion\winequality-red.csv")


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [5]:
data.isnull().sum()
data.describe()
data.info()
data.columns
data.dtypes





<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


fixed acidity           float64
volatile acidity        float64
citric acid             float64
residual sugar          float64
chlorides               float64
free sulfur dioxide     float64
total sulfur dioxide    float64
density                 float64
pH                      float64
sulphates               float64
alcohol                 float64
quality                   int64
dtype: object

In [7]:
#############ENTITY###

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [8]:
#### CONFIGURATION ####
from src.MLFlowProject.constants import *
from src.MLFlowProject.utils.common import read_yaml, create_directories
from src.MLFlowProject.entity.config_entity import DataIngestionConfig, DataValidationConfig

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation


        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir=config.unzip_data_dir,
            all_schema=self.schema.COLUMNS
        )

        return data_validation_config

In [9]:
##### COMPONENT #####
import os
import pandas as pd
from src.MLFlowProject import logger
from src.MLFlowProject.entity.config_entity import DataValidationConfig

class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_files_exist(self) -> bool:
        try:
            validation_status = True

            if not os.path.exists(self.config.unzip_data_dir):
                validation_status = False
                with open(self.config.STATUS_FILE, 'a') as f:
                    f.write(f"Validation status: {validation_status}, File is missing: {self.config.unzip_data_dir}\n")
            else:
                with open(self.config.STATUS_FILE, 'a') as f:
                    f.write(f"Validation status: {validation_status}, File exists: {self.config.unzip_data_dir}\n")

            return validation_status

        except Exception as e:
            raise e

    def validate_schema(self) -> bool:
        try:
            validation_status = True
            data = pd.read_csv(self.config.unzip_data_dir)

            # Check if all required columns exist
            for column in self.config.all_schema["COLUMNS"].keys():
                if column not in data.columns:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'a') as f:
                        f.write(f"Validation status: {validation_status}, Column is missing: {column}\n")
                else:
                    with open(self.config.STATUS_FILE, 'a') as f:
                        f.write(f"Validation status: {validation_status}, Column exists: {column}\n")

            # Check if target column exists
            if self.config.all_schema["TARGET_COLUMN"] not in data.columns:
                validation_status = False
                with open(self.config.STATUS_FILE, 'a') as f:
                    f.write(f"Validation status: {validation_status}, Target column is missing: {self.config.all_schema['TARGET_COLUMN']}\n")
            else:
                with open(self.config.STATUS_FILE, 'a') as f:
                    f.write(f"Validation status: {validation_status}, Target column exists: {self.config.all_schema['TARGET_COLUMN']}\n")

            return validation_status

        except Exception as e:
            raise e 


In [11]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_files_exist()
    data_validation.validate_schema()
except Exception as e:
    raise e

[2025-03-17 14:01:17,631: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-17 14:01:17,633: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-17 14:01:17,635: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-03-17 14:01:17,637: INFO: common: created directory at: artifacts]
[2025-03-17 14:01:17,638: INFO: common: created directory at: artifacts/data_validation]
